#### Prerequisites:
- Pandas
- glob
- subprocess

#### Introduction
This script guides you through the process of firstly, extending the initial Risk SNPs list (seed snps, PD or AD.) with Haploreg (Linkage disequilibrium) and GTex (eQTLs), and secondly, map those snps to their corresponding genes. 

In [1]:
import glob
import pandas as pd
import subprocess

1. Query Haploreg (https://pubs.broadinstitute.org/mammals/haploreg/haploreg.php) v4.1 with the initial list of seed snps provides in the folder for both PD and AD separately. They are called "PD_SNPS.txt" for Parkinson's Disease or "AD_SNPS.txt" for Alzheimers's Disease.

    - options: default, set output mode to Text, LD Threshold 0.8 
    
###### Important: Haploreg is starting to get slow when snps count exceeds ~ 80. So we first divide the the seed list into equal snps of a modarate size. Then we use it as input separately for Haploreg

In [4]:
# the path to your input file = newline separated list of seed snps
# in this example we are using PD snps. Change the path according to your study (AD or PD)
seed_snps="AD_SNPs_meemansa.txt"

In [5]:
#Import the initial snps list and divide into smaller subsets. 
#Those subsets are getting saved in the current working directory, called "snps_subset_xx".
snps_per_file = 80
smallfile=None
snps_list=[] #list of seed snps for gtex later
with open(seed_snps) as snps: # thi is the newline separated file of seed snps for PD. CHange this file according to your inpt of choice above.
    for lineno, line in enumerate(snps):
        snps_list.append(line.rstrip())
        if lineno % snps_per_file == 0:
            small_filename = 'snps_subset_{}.txt'.format(lineno + snps_per_file)
            smallfile = open(small_filename,"w")
        smallfile.write(line)
    if smallfile:
        smallfile.close()

 #### Now query Haploreg manually on their website with the smaller files separately and put them into the current working directory
 #### Follow these steps
 1. go to (https://pubs.broadinstitute.org/mammals/haploreg/haploreg.php
 2. build query
 3. build query with each subset, output format: Text, LD Threshold: 0.8
 4. Save results as "haploreg_x" with x = 1,2,3,4 etc. in the current working directory. Important to name them in that convention

In [8]:
#list of all files we created
files=glob.glob("haploreg*")

#read all files dymanically and merge them into a big dataframe
haploreg_df=pd.read_csv(files[0],delimiter = '\t')
for file in files[1:]:
    current_df=pd.read_csv(file,delimiter = '\t')
    haploreg_df = haploreg_df.append(current_df) #the final big dataframe contaiing all snps

In [9]:
#test
haploreg_df

,chr,pos_hg38,r2,D',is_query_snp,rsID,ref,alt,AFR,AMR,...,GENCODE_id,GENCODE_name,GENCODE_direction,GENCODE_distance,RefSeq_id,RefSeq_name,RefSeq_direction,RefSeq_distance,dbSNP_functional_annotation,query_snp_rsid
0,14,33974719,1,1,1,rs35833468,TAAC,T,0.35,0.52,...,ENSG00000129521.8,EGLN3,0.0,0.0,NM_022073,EGLN3,5.0,23640.0,.,rs35833468
1,3,9756586,1,1,1,rs3219012,C,T,NaN,NaN,...,ENSG00000114026.16,OGG1,0.0,0.0,NM_016828,OGG1,0.0,0.0,NSM;INT,rs3219012
2,1,171588461,1,1,1,rs2421847,A,G,0.002,0.02,...,ENSG00000117523.11,PRRC2C,0.0,0.0,NM_015172,PRRC2C,0.0,0.0,NSM,rs2421847
3,15,50699212,0.91,0.96,0,rs12592161,G,"C,T",0.19,0.3,...,ENSG00000138600.5,SPPL2A,3.0,8096.0,NM_032802,SPPL2A,3.0,8326.0,.,rs8035452
4,15,50699437,0.89,0.95,0,rs12910731,A,G,0.19,0.3,...,ENSG00000138600.5,SPPL2A,3.0,7871.0,NM_032802,SPPL2A,3.0,8101.0,.,rs8035452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1400,5,8.89102e+07,1,1,0,rs10085009,T,A,0.23,0.08,...,ENSG00000248309.1,CTC-454M9.1,0.0,0.0,NM_001131005,MEF2C,5.0,6073.0,.,rs9293506
1401,5,8.89126e+07,1,1,0,rs10056732,C,T,0.23,0.08,...,ENSG00000248309.1,CTC-454M9.1,0.0,0.0,NM_001131005,MEF2C,5.0,8537.0,.,rs9293506
1402,5,8.89138e+07,1,1,0,rs28662206,A,G,0.23,0.08,...,ENSG00000248309.1,CTC-454M9.1,0.0,0.0,NM_001131005,MEF2C,5.0,9737.0,.,rs9293506
1403,5,8.89148e+07,1,1,0,rs13357047,G,A,0.23,0.08,...,ENSG00000248309.1,CTC-454M9.1,0.0,0.0,NM_001131005,MEF2C,5.0,10732.0,.,rs9293506


In [10]:
##keep column 6 and 25 (index 5 and 24). "SNP ID" and "Gene name"
haploreg_df=haploreg_df[['rsID','GENCODE_name']]

In [11]:
#remove NaN valued rows and shady values 
haploreg_df_unique=haploreg_df.dropna(inplace=False)
print("NaN values removed")

NaN values removed


 #### Query Haploreg is done.
 ### 2.
 #### Now we we query gtex portal (http://gtexportal.org/home/index.html) to extend our mapping with eQTLs mapping. Since both Haploreg and gtex portal use genome build hg38, there's no further genome coordinate mapping needed.
 
 #### We access it through the API and fetch only Brain Tissue eQTLs

In [12]:
with open('snps_gene_eqtls.csv', 'a') as outfile: # save in csv in current working directory
    for snp in snps_list:
        #fetch brain eQTLs only
        command="curl -X GET --header \'Accept: text/html\' \'https://gtexportal.org/rest/v1/association/singleTissueEqtl?format=tsv&snpId="+snp+"&tissueSiteDetailId=Brain_Amygdala%2CBrain_Anterior_cingulate_cortex_BA24%2CBrain_Caudate_basal_ganglia%2CBrain_Cerebellar_Hemisphere%2CBrain_Cerebellum%2CBrain_Cortex%2CBrain_Frontal_Cortex_BA9%2CBrain_Hippocampus%2CBrain_Hypothalamus%2CBrain_Nucleus_accumbens_basal_ganglia%2CBrain_Putamen_basal_ganglia%2CBrain_Spinal_cord_cervical_c-1%2CBrain_Substantia_nigra&datasetId=gtex_v8\'"
        subprocess.call(command,shell=True,stdout=outfile) #that might take a few minutes

In [18]:
#open the output
eQTL_query=pd.read_csv("snps_gene_eqtls.csv",sep="\t")
#keep only snpID and gene symbol column
eQTL_query=eQTL_query[["snpId","geneSymbolUpper"]]
print (eQTL_query)

           snpId  geneSymbolUpper
0      rs1858446             GDI2
1      rs1858446             GDI2
2      rs1858446             GDI2
3      rs1858446             GDI2
4      rs1858446             GDI2
...          ...              ...
10006      snpId  geneSymbolUpper
10007      snpId  geneSymbolUpper
10008  rs1138272           DOC2GP
10009  rs1138272           DOC2GP
10010      snpId  geneSymbolUpper

[10011 rows x 2 columns]


In [19]:
#drop duplicates
eQTL_query=eQTL_query.drop_duplicates(keep="first",inplace=False)
print ("Size without duplicates:",len(eQTL_query))

Size without duplicates: 1882


In [20]:
eQTL_query.head(10) #row number 6 contains column headers

,snpId,geneSymbolUpper
0,rs1858446,GDI2
5,snpId,geneSymbolUpper
15,rs59043219,IRF6
22,rs2228479,ZNF276
23,rs2228479,MC1R
25,rs2228479,VPS9D1-AS1
27,rs2228479,SPIRE2
30,rs2228479,VPS9D1
32,rs2228479,URAHP
43,rs2228479,RP11-368I7.6


In [21]:
#we drop 2th row because it contains column headers. 
#Since we removed all duplicates, this row is the only instance of column headers within the data.
eQTL_query=eQTL_query.drop(eQTL_query.index[1]) #drop it
print("final size",len(eQTL_query))
print (eQTL_query.head(10)) # done

final size 1881
         snpId geneSymbolUpper
0    rs1858446            GDI2
15  rs59043219            IRF6
22   rs2228479          ZNF276
23   rs2228479            MC1R
25   rs2228479      VPS9D1-AS1
27   rs2228479          SPIRE2
30   rs2228479          VPS9D1
32   rs2228479           URAHP
43   rs2228479    RP11-368I7.6
56  rs10217950             CHM


In [22]:
eQTL_query

,snpId,geneSymbolUpper
0,rs1858446,GDI2
15,rs59043219,IRF6
22,rs2228479,ZNF276
23,rs2228479,MC1R
25,rs2228479,VPS9D1-AS1
...,...,...
9977,rs600491,RP11-67L3.4
9984,rs10410539,AC006277.2
9994,rs11142,PSRC1
9995,rs11142,SYPL2


### 3. 
#### Now combine Haploreg and GTex results.
####  Important: only consider eQTL SNPs, which are in Haploreg results

In [23]:
#Drop snps from the GTex reults that are NOT in Haploreg results
haploreg_ids=haploreg_df_unique.rsID.tolist() #haploreg snps ids in list
eQTL_query_haplofilter=eQTL_query[eQTL_query["snpId"].isin(haploreg_ids)] #filter Gtex reults using the haploreg ID list

In [24]:
#renaming eQTL_query_haplofilter column names to make it consistent with the Haploreg dataframe
eQTL_query_haplofilter.columns=["rsID","GENCODE_name"]

In [25]:
# finally join both Haploreg_final and eQTL_query_haplofilter 
frames = [haploreg_df_unique,eQTL_query_haplofilter]
snps_gene_haploreg_gtex = pd.concat(frames, keys=['haplo', 'gtex'])
#
snps_gene_haploreg_gtex #we are almost done... .

rsID GENCODE_name
haplo 0      rs35833468        EGLN3
      1       rs3219012         OGG1
      2       rs2421847       PRRC2C
      3      rs12592161       SPPL2A
      4      rs12910731       SPPL2A
...                 ...          ...
gtex  9977     rs600491  RP11-67L3.4
      9984   rs10410539   AC006277.2
      9994      rs11142        PSRC1
      9995      rs11142        SYPL2
      10008   rs1138272       DOC2GP

[59280 rows x 2 columns]

In [26]:
#remove duplicates from joined dataframe
snps_gene_haploreg_gtex_unique=snps_gene_haploreg_gtex.drop_duplicates(keep="first",inplace=False)
print("Size of the final Dataframe:",len(snps_gene_haploreg_gtex_unique)) #done

Size of the final Dataframe: 37477


In [27]:
# write the final dataframe to your current working directory
snps_gene_haploreg_gtex.to_csv("snps_gene_haploreg_gtex_meemansa_final.csv",index=False,header=True)